## TODO:  Put this in a pio.py library
## Create `PioModel` Class
Must have `setup()` and `predict()` methods.

## TODO:  Framework START

In [1]:
class PioModel(object):

    def __init__(self, 
                 request_transformer, 
                 response_transformer,
                 model,
                 *args,
                 **kwargs):

        self.request_transformer = request_transformer
        self.response_transformer = response_transformer
        self.model = model

    def transform_request(self, 
                          request):
        return self.request_transformer.transform_request(request)

    def transform_response(self, 
                           response):
        return self.response_transformer.transform_response(response)
    
    def setup(self,
              *args,
              **kwargs):
        pass
    
    def predict(self, 
                request,
                *args,
                **kwargs):
        self.model.predict(request)

In [2]:
class PioRequestTransformer(object):
    @classmethod
    def transform_request(self,
                          request,
                          *args,
                          **kwargs):
        pass

In [3]:
class PioResponseTransformer(object):
    @classmethod
    def transform_response(self,
                           response,
                           *args,
                           **kwargs):
        pass

## TODO:  Framework END

In [4]:
class MyModel(PioModel):

    def __init__(self, 
                 request_transformer, 
                 response_transformer,
                 model):
        PioModel.__init__(self,
                          request_transformer,
                          response_transformer,
                          model)

## Create Request Transformer
Must have a `transform_request()` method.

### Convert `json` -> `dict`

In [5]:
class MyRequestTransformer(PioRequestTransformer):     
    @classmethod
    def transform_request(request):
        import ujson
        import numpy as np
        PioRequestTransformer.transform_request(request)
        request_str = request.decode('utf-8')
        request_str = request_str.strip().replace('\n', ',')
        # surround the json with '[' ']' to prepare for conversion
        request_str = '[%s]' % request_str
        request_json = ujson.loads(request_str)

        request_transformed = ([parse_json_line(json_line) for json_line in request_json])
        return np.array(request_transformed)

    @classmethod
    def parse_json_line(json_line):
        # Note:  We're only using 1 feature in this demo
        return json_line['feature0']

## Create Response Transformer
Must have a `transform_response()` method.

### Convert `dict` -> `json`

In [6]:
class MyResponseTransformer(PioResponseTransformer):
    @classmethod
    def transform_response(response):
        # input: numpy array
        # output: list of json
        PioResponseTransformer.transform_response(response)
        return ujson.dumps(response.tolist())

## Train Model
Train and inject the following:
* `RequestTransformer`
* `ResponseTransformer`

In [ ]:
import numpy as np
from sklearn import linear_model
from sklearn import datasets

# Load the diabetes dataset
diabetes = datasets.load_diabetes()

# ONLY USING 1 FEATURE FOR THIS EXAMPLE!
# Use only one feature
diabetes_X = diabetes.data[:, np.newaxis, 2]

# Split the data into training/testing sets
diabetes_X_train = diabetes_X[:-20]
diabetes_X_test = diabetes_X[-20:]

# Split the targets into training/testing sets
diabetes_y_train = diabetes.target[:-20]
diabetes_y_test = diabetes.target[-20:]

# Create linear regression model
scikit_model = linear_model.LinearRegression()

# Train the model using the training sets
scikit_model.fit(diabetes_X_train, diabetes_y_train)

model = MyModel(MyRequestTransformer(),
                MyResponseTransformer(), 
                scikit_model)

## Pickle Model `model.pkl`

In [ ]:
import dill as pickle

model_pkl_path = 'model.pkl'

with open(model_pkl_path, 'wb') as fh:
    pickle.dump(model, fh)

In [2]:
import subprocess

output = subprocess.check_output('ls -l model.pkl',
                                 stderr=subprocess.STDOUT,
                                 shell=True)
print(output.decode('utf-8'))

CalledProcessError: Command 'ls -l model.pkl' returned non-zero exit status 2